In [38]:
import pandas as pd
# Expand the data to remove elipses and make more readable
pd.set_option('display.max_colwidth',250)
pd.set_option('display.max_columns',250)
pd.set_option('display.max_rows',1000)
import numpy as np
import requests
import nba_api
import pathlib as plib
import re
import time
from nba_api.stats.static import teams,players
from nba_api.stats.endpoints import playergamelog#, gamerotation #if getting 433 error, check each endpoint individually
from nba_api.stats.endpoints import commonteamroster, commonplayerinfo
from nba_api.stats.endpoints import leaguegamefinder, playbyplay


In [39]:
##########################
######### TEAMS ##########
##########################
# Get list of all teams
nba_teams = teams.get_teams()

# User input teams
team1_abv = input("Team 1 abv:")
team2_abv = input("Team 2 abv:")

#User input season
front_half_season = input("Season Yr:")
back_half_season = str(int(front_half_season)+1)
season_id = front_half_season + '-' + back_half_season[-2:]

# Get team_id for given teams
team1 = [team for team in nba_teams if team['abbreviation'] == team1_abv][0]
team1_id = team1['id']
team2 = [team for team in nba_teams if team['abbreviation'] == team2_abv][0]
team2_id = team2['id']

team_ids = [team1_id, team2_id]
team_ids_dict = pd.DataFrame({'ids' : team_ids})
#team_ids_dict.head()

df = leaguegamefinder.LeagueGameFinder([]).get_data_frames()[0]
#df.head()
#print(type(season_id))
#print(season_id)

In [40]:
# Query for games where the Celtics were playing
#teamgamefinder = leaguegamefinder.LeagueGameFinder(team for team in nba_teams if team['id'] == team_ids)
# The first DataFrame of those returned is what we want.
teamgames = pd.DataFrame([])
for i in range(len(team_ids)):
    df_no_tgi = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_ids[i],season_nullable=season_id).get_data_frames()[0]    

    df_no_tgi['t_game_num'] = df_no_tgi.sort_values(['TEAM_ID','GAME_ID'])['GAME_ID'].index + 1
    teamgames = pd.concat([teamgames, df_no_tgi], ignore_index=True)
    time.sleep(1)
#teamgames.head(1000)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,t_game_num
0,22022,1610612757,POR,Portland Trail Blazers,0022200894,2023-02-23,POR @ SAC,L,241,116,43,93,0.462,16,38,0.421,14,17,0.824,9,24,33,26,12,3,12,23,-17.0,64
1,22022,1610612757,POR,Portland Trail Blazers,0022200872,2023-02-14,POR vs. WAS,L,241,101,36,86,0.419,12,42,0.286,17,24,0.708,12,25,37,20,9,5,12,19,-25.0,63
2,22022,1610612757,POR,Portland Trail Blazers,0022200867,2023-02-13,POR vs. LAL,W,243,127,46,97,0.474,23,47,0.489,12,13,0.923,8,34,42,27,7,9,7,15,12.0,62
3,22022,1610612757,POR,Portland Trail Blazers,0022200843,2023-02-10,POR vs. OKC,L,241,129,42,76,0.553,16,40,0.400,29,31,0.935,9,23,32,29,3,5,18,27,-9.0,61
4,22022,1610612757,POR,Portland Trail Blazers,0022200830,2023-02-08,POR vs. GSW,W,241,125,43,78,0.551,8,27,0.296,31,32,0.969,12,35,47,25,6,3,21,16,3.0,60
5,22022,1610612757,POR,Portland Trail Blazers,0022200815,2023-02-06,POR vs. MIL,L,241,108,37,91,0.407,9,36,0.250,25,29,0.862,1,27,28,23,5,2,8,20,-19.0,59
6,22022,1610612757,POR,Portland Trail Blazers,0022200796,2023-02-04,POR @ CHI,L,242,121,44,87,0.506,14,43,0.326,19,21,0.905,7,26,33,22,3,3,9,19,-8.0,58
7,22022,1610612757,POR,Portland Trail Blazers,0022200787,2023-02-03,POR @ WAS,W,241,124,43,88,0.489,17,41,0.415,21,23,0.913,11,27,38,25,5,2,7,17,8.0,57
8,22022,1610612757,POR,Portland Trail Blazers,0022200771,2023-02-01,POR @ MEM,W,241,122,41,87,0.471,14,36,0.389,26,27,0.963,9,32,41,24,3,5,13,22,10.0,56
9,22022,1610612757,POR,Portland Trail Blazers,0022200763,2023-01-30,POR vs. ATL,W,240,129,43,79,0.544,19,40,0.475,24,27,0.889,3,29,32,24,2,7,8,21,4.0,55


In [41]:
##########################
######### PLAYERS ########
##########################
# Get list of current players on teams
team_rosters =  pd.DataFrame([])
for i in range(len(team_ids)):
    rosters = commonteamroster.CommonTeamRoster(team_id=team_ids[i]).get_data_frames()[0]

    team_rosters = pd.concat([team_rosters, rosters], ignore_index=True)
    time.sleep(1)
#team_rosters.head()
player_ids_list = team_rosters['PLAYER_ID'].unique()

In [42]:
## Player info
player_detail = pd.DataFrame(columns=['PERSON_ID','FIRST_NAME','LAST_NAME','DISPLAY_LAST_COMMA_FIRST','TEAM_NAME','TEAM_ID'])

for i in range(len(player_ids_list)):
    pd_df = commonplayerinfo.CommonPlayerInfo(player_id=player_ids_list[i]).get_data_frames()[0][['PERSON_ID','FIRST_NAME','LAST_NAME','DISPLAY_LAST_COMMA_FIRST','TEAM_NAME','TEAM_ID']]

    player_detail = pd.concat([player_detail, pd_df], ignore_index=True)
    time.sleep(1)
#player_detail.head(1000)

In [43]:
game_data = pd.DataFrame([])

for i in range(len(player_ids_list)):
    df_no_pgi = playergamelog.PlayerGameLog(player_id=player_ids_list[i]).get_data_frames()[0]
    
    df_no_pgi['p_game_num'] = df_no_pgi.sort_values(['Player_ID','Game_ID'])['Game_ID'].index + 1
    game_data = pd.concat([game_data, df_no_pgi], ignore_index=True)
    time.sleep(1)
#game_data.head(1000).sort_values(by=['Player_ID','p_game_num'], ascending= [True, True])

In [53]:
gd_pd_merge = game_data.merge(player_detail,left_on='Player_ID',right_on='PERSON_ID',how='left')
team_games_abv = teamgames[['TEAM_ID','GAME_ID','t_game_num']]
gd_pd_merge = gd_pd_merge.merge(team_games_abv,left_on=['TEAM_ID','Game_ID'],right_on=['TEAM_ID','GAME_ID'])
gd_pd_merge = gd_pd_merge[['SEASON_ID','TEAM_NAME','TEAM_ID','Player_ID','FIRST_NAME','LAST_NAME','DISPLAY_LAST_COMMA_FIRST','Game_ID','p_game_num','t_game_num','GAME_DATE','MATCHUP','WL','MIN','FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA','FT_PCT','OREB','DREB','REB','AST','STL','BLK','TOV','PF','PTS','PLUS_MINUS']]
#gd_pd_merge.head(1000)
#team_games_abv.head()
#gd_pd_merge.head(2000).sort_values(by=['Player_ID','p_game_num'], ascending= [True, True])

,SEASON_ID,TEAM_NAME,TEAM_ID,Player_ID,FIRST_NAME,LAST_NAME,DISPLAY_LAST_COMMA_FIRST,Game_ID,p_game_num,t_game_num,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS
398,22022,Trail Blazers,1610612757,203081,Damian,Lillard,"Lillard, Damian",0022200014,1,6,"OCT 19, 2022",POR @ SAC,W,37,5,18,0.278,1,8,0.125,9,9,1.000,3,1,4,8,2,0,4,0,20,-1
390,22022,Trail Blazers,1610612757,203081,Damian,Lillard,"Lillard, Damian",0022200027,2,7,"OCT 21, 2022",POR vs. PHX,W,43,12,25,0.480,5,12,0.417,12,12,1.000,1,6,7,3,0,1,4,2,41,-2
382,22022,Trail Blazers,1610612757,203081,Damian,Lillard,"Lillard, Damian",0022200037,3,8,"OCT 23, 2022",POR @ LAL,W,34,15,25,0.600,6,13,0.462,5,7,0.714,1,4,5,2,0,1,6,3,41,2
373,22022,Trail Blazers,1610612757,203081,Damian,Lillard,"Lillard, Damian",0022200051,4,9,"OCT 24, 2022",POR vs. DEN,W,35,10,16,0.625,4,7,0.571,7,8,0.875,1,5,6,8,1,0,1,2,31,24
363,22022,Trail Blazers,1610612757,203081,Damian,Lillard,"Lillard, Damian",0022200065,5,10,"OCT 26, 2022",POR vs. MIA,L,26,6,13,0.462,4,11,0.364,6,6,1.000,0,2,2,2,1,0,4,0,22,-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,22022,Trail Blazers,1610612757,1631219,John,Butler Jr.,"Butler Jr., John",0022200711,5,52,"JAN 23, 2023",POR vs. SAS,W,1,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,1,0,-1
90,22022,Trail Blazers,1610612757,1631219,John,Butler Jr.,"Butler Jr., John",0022200751,6,54,"JAN 28, 2023",POR vs. TOR,L,2,0,1,0.000,0,1,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,-1
49,22022,Trail Blazers,1610612757,1631219,John,Butler Jr.,"Butler Jr., John",0022200815,7,59,"FEB 06, 2023",POR vs. MIL,L,5,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,1,0,3
21,22022,Trail Blazers,1610612757,1631219,John,Butler Jr.,"Butler Jr., John",0022200867,8,62,"FEB 13, 2023",POR vs. LAL,W,2,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,1,0,0,0,-5


In [45]:
team_log = teamgames

filepath_tl = plib.Path(r'C:\Users\Danny\git_projects\outputs\sports_analysis_outputs\team_log.csv')  
filepath_player_stats = plib.Path(r'C:\Users\Danny\git_projects\outputs\sports_analysis_outputs\player_stats.csv')  

filepath_tl.parent.mkdir(parents=True, exist_ok=True)
filepath_player_stats.parent.mkdir(parents=True, exist_ok=True)

gd_pd_merge.to_csv(filepath_player_stats)
team_log.to_csv(filepath_tl) 
#player_team_log.head(1000).sort_values(['GAME_ID']).sort_values(['PLAYER_ID'])